<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/pyro_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyro

## An object-based toolbox for robot dynamic simulation, analysis, control and planning


This page is an introduction to the basic functionnality of pyro, an open-source python library developped at Université de Sherbrooke and hosted here: https://github.com/SherbyRobotics/pyro

<table>
  <tr>
    <th>
    <img src="https://user-images.githubusercontent.com/16725496/162986261-b3f6950b-e417-403b-8e81-81b30a542d6c.gif" alt="rocket" width="500"/>
    </th>
    <th>
    <img src="https://user-images.githubusercontent.com/16725496/163005905-ad2205b0-150d-44de-bd43-a3b31a0bf10e.gif" alt="cartpole" width="500"/>
    </th> 
  </tr>
  <tr>
    <td>
      <img src="https://user-images.githubusercontent.com/16725496/163005883-5ec9b6f8-d8ab-44b1-bc9d-ac5ca2d6b4a9.gif" alt="drone" width="500"/>
    </td>
    <td>
    <img src="https://user-images.githubusercontent.com/16725496/163005950-665132ae-c1d5-486c-8bf1-3c3fa9aa4140.gif" alt="mass-spring" width="500"/>
    </td> 
  </tr>
</table>

## Tutorial content


1.   **The Dynamic System class and basic functionnality (this page)**
2.   [Creating a custom dynamic class](https://colab.research.google.com/drive/1ILfRpL1zgiQZBOxwtbbpe0nl2znvzdWl?usp=sharing)
3.   [Closed-loop system and controllers objects](https://colab.research.google.com/drive/1mog1HAFN2NFEdw6sPudzW2OaTk_li0Vx?usp=sharing)
4.   The Linear System class (comin soon..)
4.   The Mechanical System class (coming soon..)
5.   [The Manipulator Robot class](https://colab.research.google.com/drive/1OILAhXRxM1r5PEB1BWaYtbR147Ff3gr1?usp=sharing)



## Library Architecture ##

The concept of this toolbox is a hierachy of "dynamic system" objects, from the most generic representation (any non-linear differential equations) to more system specific representations such as mechanical system (second order equations), linear state space, manipulator equations, etc. This structure is then leveraged by analysis tools, from generic tools that work for all sub-class of dynamic systems such as running simulation and phase-plane analysis, to system-specific tools that leverage specific system propreties such as modal analysis for linear sub-class.

<img width="1000" src="https://user-images.githubusercontent.com/16725496/163312294-e33d791f-9cc0-48e1-acb3-8a0ebfc0c067.jpg" class="center">

## Importing Pyro

This code show a quick exemple of

1.   Cloning pyro code source from the github repository
2.   Adding pyro folder to the python path
3.   Importing the library to the python interpreter



In [ ]:
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro


Here we import other basic python tools:

*   Numpy: the python library for linear algebra, on top of which pyro is built. 
*   Display: that is needed to show animation in the colab environment. If pyro is used locally then this is not needed.
*   Inspect: that we will use here only for printing source code in for this tutorial



In [ ]:
import numpy as np
from IPython import display
import inspect

# The Dynamic system class

Pyro is built on the concept of dynamic system classes, with standarized methods and properties. Multiple analysis and control tools are built to use the standarize methodes and properties.

**The core of a dynamic system is defined by a differential equation:** \\
$\dot{x} = f(x,u,t)$ \\
where $x$ is a state vector, $u$ is a control input vector $u$ and $t$ is the time. This function caracterize the dynamics of the system, and is used to compute the evolution of the system when a simulation is conducted.

Three (optionnal) functions can be defined, to take advantage of additionnal tools:
* **The output equation** $y = h(x,u,t)$, that compute variables that sensor would read for usage by a controller (by default $y = x$)
* **The configuration equation** $q = xu2q(x,u,t)$, that compute variables describing the configuration of the system (by default $q = x$)
* **The forward kinematic equation** $lines = forward\_kinematic\_line( q )$, that compute a graphic output for given configurations variable $q$ (by default the graphical outputs are dots with coordinates (x=q,y=0,z=0).

The following figure summarize how those function are related using a block diagram representation. 
<img width="900" src="https://user-images.githubusercontent.com/16725496/163312300-faa7fe2c-178e-4c58-ae6c-4b256fd9ab92.jpg" class="center">



## Example of basic usage

Lets first show how the object are intended to be use when all the core function are defined. Here lets load a mass-spring-damper system to exemplify.

In [ ]:
from pyro.dynamic import massspringdamper       # Here we load pyro library of mass-spring-damper

sys = massspringdamper.TwoMass()                # Here we create an instance of the TwoMass class

First to conduct simulation, we can set initial conditions, compute the trajectory, then plot the results:

In [ ]:
sys.x0 = np.array([ 1.0, 2.0, 0.0, 0.0])
sys.compute_trajectory( tf = 15.0 )
sys.plot_trajectory()

Results of a simulation are store in a trajectory object save at $sys.traj$. The trajectory object contains the time vector of x, u, y, t and methods for interpolating in those vector:

In [ ]:
print('This is the dimension of the matrix with the states trajectory: \n'  , sys.traj.x.shape      )
print('This is the the states trajectory data: \n'                          , sys.traj.x            )
print('This is the dimension of the matrix with the time  trajectory: \n'   , sys.traj.t.shape      )
print('This is the the time trajectory data: \n'                            , sys.traj.t            )
print('This find the value of x vector at time t: \n  x(t=2)= '             , sys.traj.t2x( t = 2 ) )

If foward kinematic functions have been defined we can generate an animation of this trajectory:

In [ ]:
# This would work locally in a python console
#sys.animate_simulation()

# This is the way for showing an animation on colab (we need to generate html)
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

## Adding a input to the system simulation

To include input signal $u$ to the simulation (in open-loop, for closed-loop controller see the section on this topic), a constant value can be included by changing the attribute $sys.ubar$. A time-dependent sequence can be customized by overloading the function $sys.t2u$.

In [ ]:
sys.ubar[0] = 1

sys.compute_trajectory()  # This re-compute a trajectory with new sys.ubar and sys.x0 values

ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

In [ ]:
def f(t):
  u = np.sin( 3 * t )
  return np.array([u])

sys.t2u = f

sys.compute_trajectory()

ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

The input trajectory can be plotted with this command:

In [ ]:
sys.plot_trajectory('u')

Or both state and input trajectory together with this command:

In [ ]:
sys.plot_trajectory('xu')

The dynamic system object contains attribute for the size and label of those signals that are used when generating the figures:

In [ ]:
print('The system has a state vector with dimension: ',sys.n)
print('The system has a state vector with labels: ',sys.state_label)
print('The system has a state vector with units: ',sys.state_units)

In [ ]:
print('The system has a input vector with dimension: ',sys.m)
print('The system has a input vector with labels: ',sys.input_label)
print('The system has a input vector with units: ',sys.input_units)

## Shorcuts for quick analysis
Here a few example of available analysis tool implemented as method on dynamic system object:

In [ ]:
# Phase-plane plot
sys.plot_phase_plane(0,2)

In [ ]:
# Trajectory in the phase plane
sys.plot_phase_plane_trajectory(0, 2)

In [ ]:
# Bode plot
sys.plot_linearized_bode()

In [ ]:
# Pole zero map
sys.ubar[0] = 0                 # This set back the nominal input
sys.plot_linearized_pz_map()    # This cmd linearize the system arround operating point given by sys.ubar and sys.xbar, and then compute poles

In [ ]:
# This would work locally in a python console
#sys.animate_linearized_mode( 0 )
#sys.animate_linearized_mode( 2 )

# This is the way for showing an animation on colab (we need to generate html)
ani  = sys.generate_mode_animation_html( 0 )
html = display.HTML( ani )
display.display(html)

In [ ]:
ani  = sys.generate_mode_animation_html( 2 )
html = display.HTML( ani )
display.display(html)

## Looking under the hood

Here we look at the source code of the core function of the TwoMass class that we used in our examples. The differential equation caracterizing the system has the following source code:

In [ ]:
print( inspect.getsource( sys.f ))

For this system, the differential equation used the state-space representation: 

$\dot{x} =  A x + B u$

The dynamics is defined by the matrix $A$ and $B$ that are attribute of the $sys$ class:

In [ ]:
print('A=\n',sys.A)
print('B=\n',sys.B)

Note that this class has a specific method to compute the $A,B,C,D$ matrices given mass, spring and damping parameters:

In [ ]:
print( inspect.getsource( sys.compute_ABCD ))

Most type of dynamic system class in pyro have their dynamic equation parametrized with system parameter that are attribute of the object, to allow quick modifications on the fly of the paramters.

The output equation also uses the state-space representation:

In [ ]:
print( inspect.getsource( sys.h ))

For this system the configuration variables are simply the first two states out of the four (the position of mass 1 and the position of mass 2 ):

In [ ]:
print( inspect.getsource( sys.xut2q ))

The following shows the source code of the function that compute lines given the 2 configuration variable. It is a little bit long because here we draw two moving spring and we must define many small lines.

In [ ]:
print( inspect.getsource( sys.forward_kinematic_lines ))